# Installs

In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.13.1+cu117
11.7



This may take a while

In [2]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install . 
%cd ..


Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/root/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Counting objects: 100% (26/26), done.        
remote: Compressing objects: 100% (9/9), done.        
remote: Total 82 (delta 19), reused 17 (delta 17), pack-reused 56        
Cloning into '/root/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14165, done.        
remote: Counting objects: 100% (478/478

In [ ]:
'''
If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
'''

!pip install torchsummaryx==1.1.0

  Attempting uninstall: torchsummaryx
    Found existing installation: torchsummaryX 1.3.0
    Uninstalling torchsummaryX-1.3.0:
      Successfully uninstalled torchsummaryX-1.3.0


# Imports

In [3]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat
%pip install scikit-learn

from sklearn.metrics import accuracy_score
import gc

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pandas
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Device:  cuda


In [17]:
!unzip -q ctcdecode.zip

In [22]:
%cd ctcdecode
!pip install . 
%cd ..

/root/ctcdecode
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Processing /root/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp38-cp38-linux_x86_64.whl size=19364771 sha256=995d0b77c7f8a136973241e7e57d9438a26671cb703e37994fb846fb2f7d7077
  Stored in directory: /tmp/pip-ephem-wheel-cache-wc9l5704/wheels/8f/2b/b0/cca02b3548a37d4e1eacd178a2e51b858adf4d3e7d021bfb8a
Successfully built ctcdecode
/root


In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [4]:
# TODO: Use the same Kaggle code from HW1P2
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"***","key":"******"}')

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c hw3p2asr-s24

 99% 3.71G/3.74G [00:13<00:00, 345MB/s]
100% 3.74G/3.74G [00:13<00:00, 301MB/s]


In [8]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
%cd autodl-tmp
!unzip -q 11-785-s24-hw3p2.zip
!ls
%cd ..

/root/autodl-tmp
11-785-s24-hw3p2.zip  dev-clean  test-clean  train-clean-100
/root


# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Dataset and Dataloader

In [2]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here
phonemes_sequence = ['B', 'IH', 'K', 'SH', 'AA']

# 映射音素序列到相应的ARPAbet位置
arpabet_sequence = [CMUdict_ARPAbet[phoneme] for phoneme in phonemes_sequence]

# 打印结果
print(arpabet_sequence)

['b', 'I', 'k', 'S', 'a']


In [ ]:
mfcc_dir = "/content/11-785-s24-hw3p2/train-clean-100/mfcc"
mfcc_files = sorted([file for file in os.listdir(mfcc_dir) if os.path.isfile(os.path.join(mfcc_dir, file))])
length = len(mfcc_files)
print(length)

28539


### Train Data

In [3]:
import torch
import numpy as np
import os
from torch.nn.utils.rnn import pad_sequence
import torchaudio.transforms as tat
from torch.nn import Sequential

class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes, partition = "train-clean-100"):
        """
        Initializes the audio dataset.

        Parameters:
        - root: The root directory where the dataset is stored.
        - phonemes: A list of phonemes used in the dataset.
        - partition: The dataset partition to use (e.g., 'train-clean-100').
        """
        self.phonemes = phonemes

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        assert len(self.mfcc_files) == len(self.transcript_files), "Mismatch between MFCC and transcript file counts."
        self.length = len(self.mfcc_files)

        self.mfccs, self.transcripts = [], []
        for i in range(self.length):
            # load mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
            # Cepstral normalization
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
            #  load transcript
            transcript_path  = os.path.join(self.transcript_dir, self.transcript_files[i])
            transcript = np.load(transcript_path)[1:-1]  # Exclude [SOS] and [EOS]
            # transform
            transcript_indices = np.vectorize(self.phonemes.index)(transcript)

            # append
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript_indices)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind])
        return mfcc, transcript


    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = []
        batch_transcript = []
        for mfcc, labels in batch:
            batch_mfcc.append(mfcc)
            batch_transcript.append(labels)

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = torch.tensor([len(seq) for seq in batch_mfcc])

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0)
        lengths_transcript = torch.tensor([len(seq) for seq in batch_transcript])

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        data_augmentation = Sequential(
            PermuteBlock(),
            tat.FrequencyMasking(freq_mask_param=10),   # Frequency masking
            tat.TimeMasking(time_mask_param=25),        # Time masking
            PermuteBlock()
        )
        batch_mfcc_pad_augmented = data_augmentation(batch_mfcc_pad)
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad_augmented, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)


In [4]:
class AudioDatasetValid(torch.utils.data.Dataset):
    def __init__(self, root, phonemes, partition = "dev-clean"):
        """
        Initializes the audio dataset.

        Parameters:
        - root: The root directory where the dataset is stored.
        - phonemes: A list of phonemes used in the dataset.
        - partition: The dataset partition to use (e.g., 'train-clean-100').
        """
        self.phonemes = phonemes
        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.length = len(self.mfcc_files)

        self.mfccs, self.transcripts = [], []
        for i in range(self.length):
            # load mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
            # Cepstral normalization
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

            transcript_path  = os.path.join(self.transcript_dir, self.transcript_files[i])
            transcript = np.load(transcript_path)[1:-1]  # Exclude [SOS] and [EOS]
            # transform
            transcript_indices = np.vectorize(self.phonemes.index)(transcript)

            # append
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript_indices)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind])
        return mfcc, transcript


    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = []
        batch_transcript = []
        for mfcc, labels in batch:
            batch_mfcc.append(mfcc)
            batch_transcript.append(labels)

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = torch.tensor([len(seq) for seq in batch_mfcc])

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = torch.tensor([len(seq) for seq in batch_mfcc])

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0)
        lengths_transcript = torch.tensor([len(seq) for seq in batch_transcript])

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)


### Test Data

In [5]:
# Test Dataloader
#TODO

class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self, root, partition="test-clean"):
        """
        Initializes the audio dataset for the test set.

        Parameters:
        - root: The root directory where the dataset is stored.
        - partition: The dataset partition to use, default is 'test'.
        """
        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.mfccs = []
        for mfcc_file in mfcc_files:
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_file)
            mfcc = np.load(mfcc_path)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)  # Cepstral normalization

            self.mfccs.append(mfcc)

    def __len__(self):
        return len(self.mfccs)

    def __getitem__(self, ind):
        mfcc = self.mfccs[ind]
        mfcc_tensor = torch.FloatTensor(mfcc)
        return mfcc_tensor

    def test_collate_fn(self, batch):
        # batch only mfccs
        batch_mfcc = [item for item in batch]
        lengths_mfcc = []

        # pad with pad sequence
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)

        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]

        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Config - Hyperparameters

In [10]:
root = 'autodl-tmp/'

# Feel free to add more items here
config = {
    "beam_width" : 3,
    "lr"         : 2e-3,
    "epochs"     : 50,
    "batch_size" : 64  # Increase if your device can handle it
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [73]:
# get me RAMMM!!!!
import gc
gc.collect()

1570

In [11]:
# Create objects for the dataset class
train_data = AudioDataset(root,
                          phonemes= PHONEMES,
                          partition="train-clean-100")

val_data =  AudioDatasetValid(root,
                         phonemes= PHONEMES,
                         partition="dev-clean")

test_data = AudioDatasetTest(root,
                             partition="test-clean" )


In [12]:
# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data,
                          batch_size=config['batch_size'],
                          shuffle=True,
                          num_workers = 4,
                          collate_fn=train_data.collate_fn)

val_loader = DataLoader(val_data,
                        batch_size=config['batch_size'],
                        shuffle=False,
                        num_workers = 2,
                        collate_fn=val_data.collate_fn)

test_loader = DataLoader(test_data,
                         batch_size= 1,
                         shuffle=False,
                         collate_fn=test_data.test_collate_fn)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 2620


In [14]:
# Utils for network
torch.cuda.empty_cache()
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [15]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1699, 27]) torch.Size([64, 213]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [16]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            nn.Conv1d(in_channels= 27, out_channels =64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(num_features =64),
            nn.GELU(),
            nn.Conv1d(in_channels=64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(num_features =128),
            # nn.GELU(),
            PermuteBlock())

        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = 128, hidden_size = 400, num_layers = 4,
                            bidirectional = True)

        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            nn.Linear(800, 41)
        )


        self.logSoftmax = nn.LogSoftmax(dim = -1) #TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        x = self.embedding(x)
        clamped_lx = lx.clamp(max=x.shape[1]) # to match the embedding
        x = pack_padded_sequence(x, clamped_lx, batch_first=True, enforce_sorted=False)

        x , hidden= self.lstm(x)
        output, input_sizes = pad_packed_sequence(x, batch_first = True)
        output = self.classification(output)
        out = self.logSoftmax(output)

        return out,lx

## Initialize Basic Network
(If trying out the basic Network)

In [17]:
# torch.cuda.empty_cache()
torch.cuda.empty_cache()

model = Network().to(device)
#model = nn.DataParallel(model)
#summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        # What do you return?

        return NotImplemented

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor
        return x, x_lens

### Encoder

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = #TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            # ...
            # ...
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence


        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification

        return out

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        =# TODO: Initialize Encoder
        self.decoder        =# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
input_size = 27
embed_size = 128

model = ASRModel(
    input_size  = input_size,
    embed_size  = embed_size,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential()
  (encoder): Encoder(
    (embedding): Conv1d(27, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (pBLSTMs): ModuleList(
      (0): pBLSTM(
        (blstm): LSTM(256, 128, batch_first=True, bidirectional=True)
      )
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=41, bias=True)
    )
    (softmax): LogSoftmax(dim=2)
  )
)
                                Kernel Shape     Output Shape    Params  \
Layer                                                                     
0_augmentations                            -   [32, 1645, 27]         -   
1_encoder.Conv1d_embedding      [27, 128, 3]  [32, 128, 1645]   10.496k   
2_encoder.pBLSTMs.0.LSTM_blstm             -     [20150, 256]  395.264k   
3_decoder.mlp.Linear_0            [256, 128]   [32, 822, 128]   32.896k   
4_decoder.mlp.ReLU_1                   

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations,-,"[32, 1645, 27]",NaN,NaN
1_encoder.Conv1d_embedding,"[27, 128, 3]","[32, 128, 1645]",10496.0,17055360.0
2_encoder.pBLSTMs.0.LSTM_blstm,-,"[20150, 256]",395264.0,393216.0
3_decoder.mlp.Linear_0,"[256, 128]","[32, 822, 128]",32896.0,32768.0
4_decoder.mlp.ReLU_1,-,"[32, 822, 128]",NaN,NaN
5_decoder.mlp.Linear_2,"[128, 41]","[32, 822, 41]",5289.0,5248.0
6_decoder.LogSoftmax_softmax,-,"[32, 822, 41]",NaN,NaN


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [18]:
#TODO

criterion = torch.nn.CTCLoss(blank=0, reduction='mean').to(device)
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer = torch.optim.AdamW(model.parameters(), lr= config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, beam_width = config["beam_width"], log_probs_input = True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 1, threshold=1e-2, factor = 0.5)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [34]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP=LABELS):
    # decode(self, probs, seq_lens)
    

    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output, seq_lens=output_lens)

    pred_strings = []

    for i in range(output_lens.shape[0]):
        pred = beam_results[i][0][:out_seq_len[i][0]]  # 取最可能的序列（第一个）
        pred_string = ''.join([PHONEME_MAP[n] for n in pred])
        pred_strings.append(pred_string)

    return pred_strings

import Levenshtein

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP=LABELS):
    
    print(output.shape)
    
    if output.shape[0] != 64 and output.shape[0] != 1:
        output = torch.permute(output, (1, 0, 2))
        print(f"Adjusted output shape: {output.shape}")    
    
    dist            = 0
    batch_size      = label.shape[0]
    
    pred_strings = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        label_seq = label[i][:label_lens[i]].tolist()  # 确保将tensor转换为list
        label_string = ''.join([PHONEME_MAP[n] for n in label_seq])

        # 对于pred_string，我们已经在decode_prediction中完成了转换
        pred_string = pred_strings[i]

        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size  # Average distance
    return dist



# Test Implementation

In [20]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 2936, 41])
torch.Size([2936, 64, 41]) torch.Size([64, 265])
tensor(35.5930, device='cuda:0', grad_fn=<MeanBackward0>)
253.984375


# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="0b52cf4a980eb59babb2048eeca50fdeb9de90ac") #API Key is

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huiyanx. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [39]:
run = wandb.init(
    name = "cnn_lstm_1_continue2", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [25]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [26]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [40]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 31
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = 'epoch_model.pth'#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = 'best_model.pth'#TODO set best model path

In [41]:
best = load_model(best_model_path,model,'valid_dist')
model = best[0]

In [42]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(start, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 32/50


Val:   0%|          | 0/43 [00:00<?, ?it/s]                                       

torch.Size([2936, 64, 41])
Adjusted output shape: torch.Size([64, 2936, 41])


Val:   2%|▏         | 1/43 [00:01<00:51,  1.22s/it, dist=7.7500, loss=0.5019]

torch.Size([1631, 64, 41])
Adjusted output shape: torch.Size([64, 1631, 41])


...

torch.Size([2340, 64, 41])
Adjusted output shape: torch.Size([64, 2340, 41])


Val: 100%|██████████| 43/43 [00:24<00:00,  1.94it/s, dist=5.4739, loss=0.3211]

torch.Size([3166, 15, 41])
Adjusted output shape: torch.Size([15, 3166, 41])


	Train Loss 0.0971	 Learning Rate 0.0000010
	Val Dist 5.4739%	 Val Loss 0.3211
Saved epoch model


lr,██▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆█▅▂▅▂▂▂▃▃▂▁▂▃▃▂▄
valid_dist,▇▇██▂▂▅▄▄▄▁▃▂▂▁▂▃▂▂
valid_loss,▁▂▅▅▃▅▆▇▇█▅▇▆▆▅▆▆▇▆
lr,0.0
train_loss,0.09712
valid_dist,5.47386
valid_loss,0.3211


In [30]:
run.finish()

lr,████▄▄▄▄▂▂▂▁
train_loss,▇█▆▆▄▄▃▃▂▂▁▁
valid_dist,█▇█▇▃▃▃▃▂▂▁▁
valid_loss,█▆█▇▁▁▁▂▁▂▂▂
lr,0.00025
train_loss,0.12455
valid_dist,5.57052
valid_loss,0.30276


# Generate Predictions and Submit to Kaggle

In [70]:
best = load_model(best_model_path,model,'valid_dist')
model = best[0]

In [43]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder
TEST_BEAM_WIDTH = 5

test_decoder = CTCBeamDecoder(LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)
results = []

model.eval()
print("Testing")

for data in test_loader:
    x, lx = data
    x = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    #h = torch.permute(h, (1, 0, 2))
    prediction_string = decode_prediction(h, lh, test_decoder)
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()


Testing


In [44]:
print(results[0])

-hihoptDErwUdbistufcrdInR-tRnApsAndkErAtsAndbruzdpAtetozAndf@tmAtAnpisAztubiledAldWtAndTIk-pEpRdflWRf@tAndscs-


In [45]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission_f2.csv', index = False)

In [ ]:
!kaggle competitions submit -c hw3p2asr-s24 -f submission.csv -m "I made it!"